# Rollbacks e Versionamento com Apache Iceberg

O Apache Iceberg oferece capacidades avançadas de versionamento e rollback que permitem:

## Conceitos Principais:

### 1. Snapshots
- Cada operação (INSERT, UPDATE, DELETE) cria um novo snapshot
- Snapshots são imutáveis e contêm metadados completos da tabela
- Permitem consultas pontuais no tempo (time travel)

### 2. Rollbacks
- Capacidade de reverter tabela para um snapshot anterior
- Operação rápida que apenas atualiza metadados
- Não requer reescrita de dados

### 3. Versionamento
- Histórico completo de mudanças na tabela
- Rastreabilidade de operações
- Recuperação de dados em caso de erro

## Setup do Ambiente

In [1]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession

# Sessão Spark
spark = SparkSession.builder \
    .appName("IcebergRollbacks") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Criação da Tabela de Exemplo

Vamos criar uma tabela para demonstrar os conceitos de versionamento e rollback.

In [2]:
# Criação da Tabela
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas_versioned")

spark.sql("""
    CREATE TABLE hadoop_catalog.default.vendas_versioned (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

## 2. Inserção de Dados Iniciais (Snapshot 1)

Vamos inserir os primeiros dados, criando o primeiro snapshot da tabela.

In [3]:
# Inserir dados iniciais
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_versioned VALUES
    (1, 'Produto A', 10, 100.0, DATE('2023-01-15')),
    (2, 'Produto B', 5, 200.0, DATE('2023-01-16')),
    (3, 'Produto C', 8, 150.0, DATE('2023-01-17'))
""")

DataFrame[]

### Visualizando os Dados Iniciais

In [4]:
# Visualizar dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned ORDER BY id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10|100.0|2023-01-15|
|  2|Produto B|         5|200.0|2023-01-16|
|  3|Produto C|         8|150.0|2023-01-17|
+---+---------+----------+-----+----------+



### Verificando o Primeiro Snapshot

Cada operação no Iceberg cria um snapshot. Vamos ver o snapshot criado pela inserção inicial.

In [5]:
# Ver snapshots da tabela
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas_versioned.snapshots").show(truncate=False)

+---------------+----------------------+---------+
|snapshot_id    |committed_at          |operation|
+---------------+----------------------+---------+
|513675073642359|2025-11-03 01:04:50.13|append   |
+---------------+----------------------+---------+



## 3. Adicionando Mais Dados (Snapshot 2)

Vamos adicionar mais dados, criando um segundo snapshot.

In [6]:
# Adicionar mais dados
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_versioned VALUES
    (4, 'Produto D', 12, 300.0, DATE('2023-01-18')),
    (5, 'Produto E', 7, 250.0, DATE('2023-01-19'))
""")

DataFrame[]

### Visualizando Dados Após Segunda Inserção

In [7]:
# Visualizar dados atualizados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned ORDER BY id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10|100.0|2023-01-15|
|  2|Produto B|         5|200.0|2023-01-16|
|  3|Produto C|         8|150.0|2023-01-17|
|  4|Produto D|        12|300.0|2023-01-18|
|  5|Produto E|         7|250.0|2023-01-19|
+---+---------+----------+-----+----------+



### Verificando os Snapshots Existentes

Agora devemos ter dois snapshots: um para cada operação de inserção.

In [8]:
# Ver todos os snapshots
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas_versioned.snapshots ORDER BY committed_at").show(truncate=False)

+-------------------+----------------------+---------+
|snapshot_id        |committed_at          |operation|
+-------------------+----------------------+---------+
|513675073642359    |2025-11-03 01:04:50.13|append   |
|2421557506823078562|2025-11-03 01:05:01.82|append   |
+-------------------+----------------------+---------+



## 4. Atualizando Dados (Snapshot 3)

Vamos fazer uma atualização que pode ser considerada "problemática" e que queremos reverter depois.

In [9]:
# Fazer uma atualização "problemática"
spark.sql("""
    UPDATE hadoop_catalog.default.vendas_versioned 
    SET preco = preco * 10 
    WHERE id <= 3
""")

DataFrame[]

### Visualizando Dados Após Atualização "Problemática"

Os preços dos produtos 1, 2 e 3 foram multiplicados por 10 (erro simulado).

In [10]:
# Ver dados após atualização problemática
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned ORDER BY id").show()

+---+---------+----------+------+----------+
| id|  produto|quantidade| preco|data_venda|
+---+---------+----------+------+----------+
|  1|Produto A|        10|1000.0|2023-01-15|
|  2|Produto B|         5|2000.0|2023-01-16|
|  3|Produto C|         8|1500.0|2023-01-17|
|  4|Produto D|        12| 300.0|2023-01-18|
|  5|Produto E|         7| 250.0|2023-01-19|
+---+---------+----------+------+----------+



### Verificando Todos os Snapshots

Agora temos três snapshots: duas inserções e uma atualização.

In [11]:
# Ver todos os snapshots após update
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas_versioned.snapshots ORDER BY committed_at").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|513675073642359    |2025-11-03 01:04:50.13 |append   |
|2421557506823078562|2025-11-03 01:05:01.82 |append   |
|5583770746944098475|2025-11-03 01:05:13.069|overwrite|
+-------------------+-----------------------+---------+



## 5. Time Travel - Consultando Snapshot Anterior

Antes de fazer o rollback, vamos consultar como os dados estavam no snapshot anterior para confirmar que queremos voltar para lá.

In [12]:
# Consultar dados do snapshot anterior (antes da atualização problemática)
# Primeiro, vamos pegar o ID do segundo snapshot
snapshots = spark.sql("SELECT snapshot_id FROM hadoop_catalog.default.vendas_versioned.snapshots ORDER BY committed_at")
second_snapshot = snapshots.collect()[1][0]  # Segundo snapshot (índice 1)
print(f"ID do segundo snapshot: {second_snapshot}")

# Consultar dados como estavam no segundo snapshot
spark.sql(f"""
    SELECT * FROM hadoop_catalog.default.vendas_versioned 
    VERSION AS OF {second_snapshot}
    ORDER BY id
""").show()

ID do segundo snapshot: 2421557506823078562
+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10|100.0|2023-01-15|
|  2|Produto B|         5|200.0|2023-01-16|
|  3|Produto C|         8|150.0|2023-01-17|
|  4|Produto D|        12|300.0|2023-01-18|
|  5|Produto E|         7|250.0|2023-01-19|
+---+---------+----------+-----+----------+



## 6. Executando Rollback

Agora vamos fazer o rollback para o snapshot anterior, desfazendo a atualização problemática.

In [13]:
# Fazer rollback para o segundo snapshot
spark.sql(f"""
    CALL hadoop_catalog.system.rollback_to_snapshot(
        'hadoop_catalog.default.vendas_versioned', 
        {second_snapshot}
    )
""")

DataFrame[previous_snapshot_id: bigint, current_snapshot_id: bigint]

### Verificando Dados Após Rollback

Os dados devem estar como estavam antes da atualização problemática.

In [14]:
# Verificar dados após rollback
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned ORDER BY id").show()

+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10|100.0|2023-01-15|
|  2|Produto B|         5|200.0|2023-01-16|
|  3|Produto C|         8|150.0|2023-01-17|
|  4|Produto D|        12|300.0|2023-01-18|
|  5|Produto E|         7|250.0|2023-01-19|
+---+---------+----------+-----+----------+



### Verificando Snapshots Após Rollback

O rollback cria um novo snapshot que aponta para o estado anterior.

In [15]:
# Ver snapshots após rollback
spark.sql("SELECT snapshot_id, committed_at, operation FROM hadoop_catalog.default.vendas_versioned.snapshots ORDER BY committed_at").show(truncate=False)

+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|513675073642359    |2025-11-03 01:04:50.13 |append   |
|2421557506823078562|2025-11-03 01:05:01.82 |append   |
|5583770746944098475|2025-11-03 01:05:13.069|overwrite|
+-------------------+-----------------------+---------+



## 7. Histórico Completo da Tabela

Podemos ver o histórico completo de operações na tabela.

In [16]:
# Ver histórico completo da tabela
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_versioned.history ORDER BY made_current_at").show(truncate=False)

+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-11-03 01:04:50.13 |513675073642359    |null               |true               |
|2025-11-03 01:05:01.82 |2421557506823078562|513675073642359    |true               |
|2025-11-03 01:05:13.069|5583770746944098475|2421557506823078562|false              |
|2025-11-03 01:05:20.929|2421557506823078562|513675073642359    |true               |
+-----------------------+-------------------+-------------------+-------------------+



## 8. Rollback por Timestamp

Além de rollback por snapshot ID, também podemos fazer rollback por timestamp.

In [17]:
# Exemplo de rollback por timestamp (comentado para não afetar o exemplo)
# from datetime import datetime, timedelta
# 
# # Rollback para 10 minutos atrás
# ten_minutes_ago = datetime.now() - timedelta(minutes=10)
# timestamp_str = ten_minutes_ago.strftime('%Y-%m-%d %H:%M:%S')
# 
# spark.sql(f"""
#     CALL hadoop_catalog.system.rollback_to_timestamp(
#         'hadoop_catalog.default.vendas_versioned', 
#         TIMESTAMP '{timestamp_str}'
#     )
# """)

print("Rollback por timestamp permite voltar a um ponto específico no tempo")
print("Útil quando você sabe quando o problema ocorreu, mas não o snapshot exato")

Rollback por timestamp permite voltar a um ponto específico no tempo
Útil quando você sabe quando o problema ocorreu, mas não o snapshot exato


## Resumo dos Benefícios

### Vantagens do Sistema de Rollback do Iceberg:

1. **Operação Rápida**: Rollbacks são operações de metadados, não requerem reescrita de dados
2. **Granularidade**: Pode voltar para qualquer snapshot ou timestamp específico
3. **Segurança**: Dados antigos não são perdidos, apenas "escondidos"
4. **Auditoria**: Histórico completo de todas as operações
5. **Recuperação**: Facilita recuperação de erros humanos ou de sistema

### Casos de Uso Comuns:

- Desfazer atualizações incorretas
- Recuperar de corrupção de dados
- Voltar a um estado conhecido bom
- Testes e desenvolvimento
- Compliance e auditoria